In [78]:
import os
import requests
from openai import OpenAI
import ipywidgets as widgets
from dotenv import load_dotenv
from IPython.display import display, Markdown, update_display, Javascript

In [79]:
MODEL_GPT   = 'gpt-4o-mini'
MODEL_LLAMA = 'llama3.2'

In [85]:
load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

if api_key and api_key.startswith('sk-proj-') and len(api_key)>10:
    print("API key looks good so far")
else:
    print("There might be a problem with your API key? Please visit the troubleshooting notebook!")
openai = OpenAI()
ollama_via_openai = OpenAI(base_url='http://localhost:11434/v1', api_key='ollama')

API key looks good so far


In [81]:
system_prompt = "You are an intelligent and patient technical tutor specialized in computer science, software engineering, and data science. " \
"Your goal is to explain technical concepts in a clear, concise, and engaging way — as if you are teaching a motivated student who is learning " \
"but may not yet be an expert. " \
"When the user asks a question:\n\n" \
"- Break it down step-by-step.\n" \
"- Use simple examples or analogies if needed.\n" \
"- Clarify jargon or advanced terms.\n" \
"- If relevant, provide code snippets or diagrams in plain text.\n" \
"- Be accurate and don't hallucinate — say \"I don't know\" if you're unsure.\n\n" \
"You are not just answering; you are teaching. Always aim to help the user truly understand the underlying concept."

In [82]:
def get_messages(question):
    return [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": question}
    ]

In [96]:
def get_gpt_response(messages, output_widget):
    stream = openai.chat.completions.create(
        model = MODEL_GPT,
        messages = messages,
        stream = True
    )
    
    response = ""
    for chunk in stream:
        content = chunk.choices[0].delta.content or ''
        content = content.replace("``", "").replace("markdown", "")
        response += content
        output_widget.value = f"<b style='color:#1e8449;'>Tutor:</b> {response}"

In [87]:
def get_ollama_response(messages, output_widget):
    stream = ollama_via_openai.chat.completions.create(
        model = MODEL_LLAMA,
        messages = messages,
        stream = True
    )
    
    response = ""
    for chunk in stream:
        content = chunk.choices[0].delta.content or ''
        content = content.replace("``", "").replace("markdown", "")
        response += content
        output_widget.value = f"<b style='color:#1e8449;'>Tutor:</b> {response}"

In [97]:
heading = widgets.HTML("""
    <h2 style="text-align:center; color:#2e86de; font-family:'Segoe UI', Tahoma, Geneva, Verdana, sans-serif;">
        🤖 AI Tutor
    </h2>
""")

In [98]:
chat_log = widgets.VBox()
chat_scroll = widgets.Box([chat_log], layout=widgets.Layout(
    border='1px solid #dcdcdc',
    border_radius='8px',
    height='400px',
    overflow_y='auto',
    padding='10px',
    background_color='#fefefe',
    box_shadow='0 2px 4px rgba(0,0,0,0.1)'
))

In [99]:
model_selector = widgets.Dropdown(
    options=[('GPT', 'gpt'), ('Ollama', 'ollama')],
    value='gpt',
    description='Model:',
    layout=widgets.Layout(width='200px'),
    style={'description_width': 'initial'}
)


In [108]:
question_input = widgets.Textarea(
    placeholder='Type your technical question here...',
    layout=widgets.Layout(width='99.8%', height='70px', border='1px solid #ccc', border_radius='6px'),
    style={
        'description_width': 'initial',
        'font_family': 'Segoe UI',
        'font_size': '14px'
    }
)

In [101]:
ask_button = widgets.Button(
    description="Ask",
    button_style='',
    layout=widgets.Layout(width='100px', height='40px'),
    style={
        'button_color': '#2e86de',
        'font_weight': 'bold',
        'font_size': '14px',
        'text_color': '#ffffff'
    }
)


In [110]:

input_controls = widgets.HBox([model_selector, ask_button], layout=widgets.Layout(justify_content='space-between', margin='5px 0'))
input_area = widgets.VBox([question_input, input_controls])


ui_container = widgets.VBox([heading, chat_scroll, input_area])
display(ui_container)


def scroll_to_bottom():
    display(Javascript("""
        const out = document.querySelectorAll('.output_subarea')[document.querySelectorAll('.output_subarea').length - 1];
        if (out) out.scrollTop = out.scrollHeight;
    """))

def on_submit(_):
    question = question_input.value.strip()
    if not question:
        return
    if question.lower() == 'exit':
        chat_log.children += (widgets.HTML("<b>Goodbye! 👋</b>"),)
        return

    question_input.value = ""  # Clear input
    user_msg = widgets.HTML(f"<b style='color:#2c3e50;'>You:</b> {question}")
    chat_log.children += (user_msg,)

    messages = get_messages(question)
    response_widget = widgets.HTML(value="<b style='color:#1e8449;'>Tutor:</b> ")
    chat_log.children += (response_widget,)

    scroll_to_bottom()

    if model_selector.value == 'gpt':
        get_gpt_response(messages, response_widget)
    else:
        get_ollama_response(messages, response_widget)

ask_button.on_click(on_submit)


def handle_enter(key):
    if key['key'] == 'Enter' and not key['shiftKey']:
        on_submit(None)
        key['preventDefault']()

question_input.on_msg(handle_enter)